<a href="https://colab.research.google.com/github/Naaastenka/Project1/blob/main/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
con = sqlite3.connect(':memory:')

In [ ]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [ ]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [ ]:
pd.read_sql(
     '''
    SELECT CustomerName, Country, Address
    FROM Customers
    WHERE Country IN ('Germany', 'France') OR City = 'Madrid'
    ''',
    con,
)

2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [ ]:
pd.read_sql(
    '''
    SELECT Country, COUNT(*) as CustomerCount
    FROM Customers
    GROUP BY Country
    ORDER BY CustomerCount DESC
    LIMIT 3
    ''',
    con,
)

3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [ ]:
pd.read_sql(
    '''
    SELECT s.ShipperName, o.ShippedDate
    FROM Orders o
    JOIN Shippers s ON o.ShipperID = s.ShipperID
    ORDER BY o.ShippedDate
    LIMIT 1 OFFSET 9
    ''',
)

4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [ ]:
pd.read_sql(
    '''
    SELECT p.ProductName, od.UnitPrice
    FROM OrderDetails od
    JOIN Products p ON od.ProductID = p.ProductID
    WHERE od.OrderID = (
        SELECT od.OrderID
        FROM OrderDetails od
        GROUP BY od.OrderID
        ORDER BY SUM(od.UnitPrice * od.Quantity) DESC
        LIMIT 1
    )
    ''',
    con,
)

5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [ ]:
pd.read_sql(
    '''
    SELECT p.ProductName, od.Quantity
    FROM OrderDetails od
    JOIN Products p ON od.ProductID = p.ProductID
    WHERE od.ProductID = (
        SELECT od.ProductID
        FROM OrderDetails od
        GROUP BY od.ProductID
        ORDER BY SUM(od.Quantity) DESC
        LIMIT 1
    )
    ''',
    con,
)

6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [ ]:
pd.read_sql(
    '''
    SELECT s.SupplierName, s.Country, s.ContactName, s.Phone, COUNT(*) as OrderCount
    FROM Suppliers s
    JOIN Products p ON s.SupplierID = p.SupplierID
    JOIN OrderDetails od ON p.ProductID = od.ProductID
    GROUP BY s.SupplierID
    ORDER BY OrderCount DESC
    LIMIT 5
    ''',
    con,
)

7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [ ]:
pd.read_sql(
    '''
    SELECT c.Country, cat.CategoryName, SUM(od.UnitPrice * od.Quantity) AS TotalAmount
    FROM OrderDetails od
    JOIN Orders o ON od.OrderID = o.OrderID
    JOIN Customers c ON o.CustomerID = c.CustomerID
    JOIN Products p ON od.ProductID = p.ProductID
    JOIN Categories cat ON p.CategoryID = cat.CategoryID
    WHERE c.Country = 'Brazil'
    GROUP BY cat.CategoryID
    ORDER BY TotalAmount DESC
    LIMIT 1
    ''',
    con,
)

8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [ ]:
pd.read_sql(
    '''
    WITH OrderCosts AS (
        SELECT o.OrderID, SUM(od.UnitPrice * od.Quantity) AS TotalCost
        FROM Orders o
        JOIN OrderDetails od ON o.OrderID = od.OrderID
        JOIN Customers c ON o.CustomerID = c.CustomerID
        WHERE c.Country = 'USA'
        GROUP BY o.OrderID
    )
    SELECT MAX(TotalCost) - MIN(TotalCost) AS CostDifference
    FROM OrderCosts
    ''',
    con,
)

9.Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [ ]:
pd.read_sql(
    '''
    WITH YoungestEmployees AS (
        SELECT EmployeeID, FirstName || ' ' || LastName AS FullName
        FROM Employees
        ORDER BY BirthDate DESC
        LIMIT 3
    )
    SELECT ye.FullName, COUNT(o.OrderID) AS OrderCount
    FROM YoungestEmployees ye
    LEFT JOIN Orders o ON ye.EmployeeID = o.EmployeeID
    GROUP BY ye.EmployeeID, ye.FullName
    ''',
    con,
)

10. Сколько банок крабового мяса всего было заказано.

In [ ]:
pd.read_sql(
    '''
    SELECT SUM(od.Quantity) AS TotalQuantity
    FROM OrderDetails od
    JOIN Products p ON od.ProductID = p.ProductID
    WHERE p.ProductName = 'Boston Crab Meat'
    ''',
    con,
)